# Exploring a (maybe) new way of formulating and solving the Model Predictive Control (MPC) problem

In [ ]:
import torch as th
import numpy as np
np.set_printoptions(precision=5, linewidth=120)
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# close all figures
plt.close('all')

In [ ]:
# define the simple cruise control model
mass = 1e3 # [kg] mass of the car
damp = 10 # [Ns/m] damping coefficient
ms2kmh = 3.6 # [m/s] to [km/h]
dstrb = mass * 9.81 * np.sin(np.deg2rad(5)) # [N] disturbance force (slope of 5 degrees)
start_dstrb = 150 # [s] start of the disturbance

ref = 50 # [km/h] reference speed

u_max = 1e8 # [N] maximum control input

dt = .1 # [s] time step simulation
T = 500 # [s] simulation time

Ts = 1.0 # [s] sampling time 

tc = np.linspace(0, T, int(T/dt))
td = np.linspace(0, T, int(T/Ts))

print(f'disturbance force: {dstrb:.2f} N')

In [ ]:
# state space formulation
n = 1 # dimension of the state
m = 1 # dimension of the input
p = 1 # dimension of the output
q = 1 # dimension of the disturbance

A = np.array([[-damp/mass]]) # state matrix (nxn)
B = np.array([[1/mass]]) # input matrix (nxm)
C = np.array([[ms2kmh]]) # output matrix (pxn)
D = np.array([[0.]]) # feedforward matrix (pxm)
M = np.array([[-1/mass]]) # disturbance matrix (nxq)

assert A.shape == (n, n), f'A.shape: {A.shape}'
assert B.shape == (n, m), f'B.shape: {B.shape}'
assert C.shape == (p, n), f'C.shape: {C.shape}'
assert D.shape == (p, m), f'D.shape: {D.shape}'
assert M.shape == (n, q), f'M.shape: {M.shape}'

In [ ]:
#discretize using scipy
from scipy.signal import cont2discrete, lti, dlti, dstep
lti_sys = lti(A,B,C,D)
lti_sysd = cont2discrete((A,B,C,D), Ts, method='zoh') # zero order hold

# plot
plt.figure(figsize=(10, 3))
t1, y1 = lti_sys.step(T=tc)
plt.plot(tc, y1, label='continuous')
for method in ['zoh', 'bilinear', 'euler', 'backward_diff', 'foh', 'impulse']:
    tmp_sys = cont2discrete((A,B,C,D), Ts, method=method)
    t2, y2 = dstep(tmp_sys, t=td)
    plt.step(t2, y2[0], label=method)
plt.legend()
plt.title('Discretization methods')
plt.tight_layout()
plt.show()

In [ ]:
# now work with the discretized system
A, B, C, D, Ts = lti_sysd
print(f'A: {A}, B: {B}, C: {C}, D: {D}, Ts: {Ts}')

In [ ]:
#helper functions
# function to plot step response and relative input (assume 1d input and output)
def plot_step(t,u,y,ref=None):
    t, u, y = t.reshape(-1), u.reshape(-1), y.reshape(-1)
    assert t.shape == u.shape == y.shape
    plt.figure(figsize=(10, 5))
    plt.subplot(211)
    if ref is not None: plt.plot(t, ref, 'r--')
    plt.step(t, y)
    plt.title('Step response')
    plt.subplot(212)
    plt.step(t, u)
    plt.title('Input')
    plt.tight_layout()
    plt.show()

# function to simulate the system
def sim_step(x, u, d, A, B, C):
    assert x.shape == (n, 1), f'x.shape: {x.shape}'
    assert u.shape == (m, 1), f'u.shape: {u.shape}'
    assert A.shape == (n, n), f'A.shape: {A.shape}'
    assert B.shape == (n, m), f'B.shape: {B.shape}'
    assert C.shape == (p, n), f'C.shape: {C.shape}'
    x1 = A @ x + B @ u + M @ d
    y = C @ x
    assert x1.shape == (n, 1), f'x1.shape: {x1.shape}'
    assert y.shape == (p, 1), f'y.shape: {y.shape}'
    return x1, y

In [ ]:
# simulate a step response using a for loop
xs, ys = np.zeros((len(td), 1, m)), np.zeros((len(td), 1, n))
us = 900 * np.heaviside(td-1, 1).reshape(-1, 1, m)
ds = dstrb * np.heaviside(td-start_dstrb, 1).reshape(-1, 1, q)
for i in range(1, len(td)):
    xs[i], ys[i] = sim_step(xs[i-1], us[i-1], ds[i-1], A, B, C)

plot_step(td, us, ys)

In [ ]:
# test a simple PD controller
class PID():
    def __init__(self, Kp, Ki, Kd, Ts):
        self.Kp, self.Ki, self.Kd, self.Ts = Kp, Ki, Kd, Ts
        self.e_prev = 0 # previous error
        self.int_e = 0 # integral of error
    def get_control(self, r, y):
        e = r - y
        self.int_e += e * self.Ts
        u = self.Kp * e + self.Ki * self.int_e + self.Kd * (e - self.e_prev) / self.Ts
        self.e_prev = e
        return u

Kp = 50
Ki = 0 # 3
Kd = .3

pid = PID(Kp, Ki, Kd, Ts)

# simulate a step response using a for loop
xs, ys = np.zeros((len(td), 1, m)), np.zeros((len(td), 1, n))
us = np.zeros((len(td), 1, m))
rs = ref * np.ones((len(td), p))
ds = dstrb * np.heaviside(td-start_dstrb, 1).reshape(-1, 1, q)
for i in range(1, len(td)):
    us[i] = pid.get_control(rs[i-1], ys[i-1,0]).reshape(1, 1)
    xs[i], ys[i] = sim_step(xs[i-1], us[i], ds[i-1], A, B, C)
    
plot_step(td, us, ys, rs)

In [ ]:
# test MPC, standard form
N = 10 # prediction horizon (number of steps)

Q = np.array([[1]]) # state cost (n x n)
R = np.array([[1]]) # input cost (m x m)
S = np.array([[0]]) # terminal state cost (n x n)

# input constraint: u_min <= u <= u_max
F = np.array([[1], [-1]]) # inequality constraint matrix (2m x m)
f = np.array([[u_max], [u_max]]) # inequality constraint vector (2m x 1)


In [ ]:
# creating the condensed matrices
import numpy as np
from numpy import kron, eye, zeros, ones

def create_condensed_matrices(A, B, C, M, Q, R, S, F, f, N):
    """
    Create condensed matrices for Model Predictive Control.
    
    Parameters:
    -----------
    A, B, C : ndarray
        State space matrices
    M : ndarray
        Disturbance matrix
    Q, R, S : ndarray
        State, input, terminal cost matrices
    F, f : ndarray
        Inequality constraints
    N : int
        Prediction horizon
        
    Returns:
    --------
    tuple:
        (cAC, cBC, cMC, cQ, cR, cF, cf, x_term_r)
        Condensed matrices for the MPC problem
    """
    # Get dimensions
    n, m = A.shape[0], B.shape[1]
    p, q = C.shape[0], M.shape[1]
    
    # Check dimensions
    assert A.shape == (n, n), f'A.shape: {A.shape}'
    assert B.shape == (n, m), f'B.shape: {B.shape}'
    assert C.shape == (p, n), f'C.shape: {C.shape}'
    assert M.shape == (n, q), f'M.shape: {M.shape}'
    assert Q.shape == (n, n), f'Q.shape: {Q.shape}'
    assert R.shape == (m, m), f'R.shape: {R.shape}'
    assert S.shape == (n, n), f'S.shape: {S.shape}'

    # Create standard condensed matrices
    cQ = kron(eye(N), Q)
    cR = kron(eye(N), R)
    cF = kron(eye(N), F)
    cf = kron(ones((N, 1)), f)

    # Pre-allocate matrices
    cAC = zeros((N*p, n))
    cBC = zeros((N*p, N*m))
    cMC = zeros((N*p, N*q))

    # Fill matrices
    for i in range(1, N+1):
        row_idx = (i-1)*p
        
        # cAC
        cAC[row_idx:row_idx+p, :] = C @ np.linalg.matrix_power(A, i)
        
        # cBC and cMC
        for j in range(1, N+1):
            col_idx_B = (j-1)*m
            col_idx_M = (j-1)*q
            
            if j <= i:
                A_power = np.linalg.matrix_power(A, i-j)
                cBC[row_idx:row_idx+p, col_idx_B:col_idx_B+m] = C @ A_power @ B
                cMC[row_idx:row_idx+p, col_idx_M:col_idx_M+q] = C @ A_power @ M
    
    # Add terminal cost
    # Update cQ with terminal cost
    cQ = np.block([
        [cQ[:(N-1)*p, :(N-1)*p], zeros(((N-1)*p, n))],
        [zeros((n, (N-1)*p)), S]
    ])
    
    # Create new matrices with terminal cost
    cAC_new = zeros((N*p, n))
    cAC_new[:(N-1)*p, :] = cAC[:(N-1)*p, :]
    cAC_new[(N-1)*p:, :] = np.linalg.matrix_power(A, N)
    cAC = cAC_new
    
    # Update cBC
    cBC_new = zeros((N*p, N*m))
    cBC_new[:(N-1)*p, :(N-1)*m] = cBC[:(N-1)*p, :(N-1)*m]
    
    # Fill in the bottom row for cBC
    for j in range(1, N+1):
        col_idx = (j-1)*m
        if j <= N:
            cBC_new[(N-1)*p:(N-1)*p+n, col_idx:col_idx+m] = np.linalg.matrix_power(A, N-j) @ B
    cBC = cBC_new
    
    # Update cMC
    cMC_new = zeros((N*p, N*q))
    cMC_new[:(N-1)*p, :(N-1)*q] = cMC[:(N-1)*p, :(N-1)*q]
    
    # Fill in the bottom row for cMC
    for j in range(1, N+1):
        col_idx = (j-1)*q
        if j <= N:
            cMC_new[(N-1)*p:(N-1)*p+n, col_idx:col_idx+q] = np.linalg.matrix_power(A, N-j) @ M
    cMC = cMC_new
    
    # Terminal reference
    x_term_r = zeros((n, 1))
    
    # Return tuple of matrices
    return cAC, cBC, cMC, cQ, cR, cF, cf, x_term_r

In [ ]:
# test the condensed matrices function
cAC, cBC, cMC, cQ, cR, cF, cf, x_term_r = create_condensed_matrices(A, B, C, M, Q, R, S, F, f, N)

print(f'cAC: {cAC.shape}, cBC: {cBC.shape}, cMC: {cMC.shape}, cQ: {cQ.shape}, cR: {cR.shape}, cF: {cF.shape}, cf: {cf.shape}, x_term_r: {x_term_r.shape}')

print(f'cAC:\n{cAC}')
print(f'cBC:\n{cBC}')
print(f'cMC:\n{cMC}')
print(f'cQ:\n{cQ}')
print(f'cR:\n{cR}')
print(f'cF:\n{cF}')
print(f'cf:\n{cf}')
print(f'x_term_r:\n{x_term_r}')